In [ ]:
%pip install huggingface_hub

In [6]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

base_model ='TinyLlama/TinyLlama_v1.1'


model = AutoModelForCausalLM.from_pretrained(base_model,
                                            device_map="auto",
                                            torch_dtype=torch.float16,
                                            load_in_8bit=False)

In [7]:
from transformers.trainer_utils import get_last_checkpoint
from peft import PeftModel

model_path = get_last_checkpoint("Checkpoint")
peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

In [8]:
model = peft_model.merge_and_unload()

In [9]:
print(type(model))

<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [ ]:
from huggingface_hub import login
login(token="seu_token_de_acesso")

In [13]:
model.push_to_hub("TinyLlama_v1.1-qlora-finetunined-UFV")

model.safetensors: 100%|██████████| 2.20G/2.20G [02:26<00:00, 15.0MB/s]


CommitInfo(commit_url='https://huggingface.co/Sparcos/TinyLlama_v1.1-qlora-finetunined-UFV/commit/b0d04f935710e89db207a73074256cdb3e1b77a6', commit_message='Upload LlamaForCausalLM', commit_description='', oid='b0d04f935710e89db207a73074256cdb3e1b77a6', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
from transformers.trainer_utils import get_last_checkpoint
from transformers import AutoTokenizer
max_length = 2048 # valor usado no treinamento do tinyllama

model_path = get_last_checkpoint("Checkpoint")
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                        trust_remote_code=True,
                                        max_length= max_length, 
                                        truncation=True)
    
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [15]:
tokenizer.push_to_hub("TinyLlama_v1.1-qlora-finetunined-UFV")

CommitInfo(commit_url='https://huggingface.co/Sparcos/TinyLlama_v1.1-qlora-finetunined-UFV/commit/92f734aa53ee0cdd5c99fbaa3e7bcaa064388366', commit_message='Upload tokenizer', commit_description='', oid='92f734aa53ee0cdd5c99fbaa3e7bcaa064388366', pr_url=None, pr_revision=None, pr_num=None)